# 0. Importing packages & Defining methods

## 1) Importing packages

In [1]:
from keras.layers import Input, Dense    #using set model component
from keras.models import Model    #using set model 
from keras.utils import plot_model    #show model structure
from keras import layers as Layer
import keras 
from lib import dataProcess as dp
import pandas as pd
from pandas import DataFrame as df
from keras.callbacks import EarlyStopping
import pydot
# prediction


Using TensorFlow backend.


## 2) Defining methods

In [2]:
def check_correct(predict, y):
    result = {}
    result['resistant-correct'] = 0
    result['resistant-wrong'] = 0
    result['sensitive-correct'] = 0
    result['sensitive-wrong'] = 0

    for i in range(len(predict)) :
        if predict[i] == y[i] :
            if y[i] == 0 :
                result['sensitive-correct'] += 1
            else :
                result['resistant-correct'] += 1
        else :
            if y[i] == 0 :
                result['sensitive-wrong'] += 1
            else :
                result['resistant-wrong'] += 1

    #for result_k, result_v in result.items():
    #    print(result_k +" : "+ str(result_v))
    sensitivity=result['resistant-correct']/(result['resistant-correct']+result['resistant-wrong'])
    specificity=result['sensitive-correct']/(result['sensitive-correct']+result['sensitive-wrong'])
    #print("Sensitivity :", sensitivity)
    #print("Specificity :", specificity)
    return sensitivity, specificity


# 1. Reading data & Preprocessing

## 1) Path selection

In [19]:
methods = ["old", "new"]
genesets = ["gene_set1", "gene_set2"]
limma_options = ["before_limma", "limma"]

limma_option = limma_options[int(input("choose the limma_option (1 or 2):\n[1] before_limma\n[2] limma\n"))-1]
print("you select "+limma_option+"\n")
method = methods[int(input("choose the method (1 or 2):\n[1] old\n[2] new\n"))-1]
print("you select "+method+"\n")
geneset = genesets[int(input("choose the geneset (1 or 2):\n[1] gene_set1\n[2] gene_set2\n"))-1]
print("you select "+geneset+"\n")

path = "../../Data/"+method+"_set/"
save_model_path = "../models/"+method+"_set/"
save_result_path = "../results/"+method+"_set/"

print("###################################################################")
print("path: "+path)
print("save_model_path: "+save_model_path)
print("save_result_path: "+save_result_path)

choose the limma_option (1 or 2):
[1] before_limma
[2] limma
2
you select limma

choose the method (1 or 2):
[1] old
[2] new
2
you select new

choose the geneset (1 or 2):
[1] gene_set1
[2] gene_set2
1
you select gene_set1

###################################################################
path: ../../Data/new_set/
save_model_path: ../models/new_set/
save_result_path: ../results/new_set/


## 2) Read data files & Dividing into train / test

In [20]:
train_data=pd.read_csv(path+limma_option+"_TCGA_inter_nonmucinous_"+method+"_"+geneset+"_data.csv")
test_data=pd.read_csv(path+limma_option+"_TCGA_inter_mucinous_"+method+"_"+geneset+"_data.csv")
cli_data=pd.read_csv(path+limma_option+"_TCGA_inter_clin_"+method+"_"+geneset+"_data.csv")

#train_data=train_data.sample(frac=1)
train_x, train_y = dp.divide_xy_test(train_data, 'result',  1, -1)
test_x, test_y=dp.divide_xy_test(test_data, 'result',  1, -1)
cli_x = cli_data.iloc[:, 1:].as_matrix()
train_y1 = dp.one_hot_encoder_train(train_y)
test_y1 = dp.one_hot_encoder_test(train_y,test_y)

C:\test\mucinous_training\TCGA_mucinous_DNN_space\codes\lib\dataProcess.py:133: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  ydata = data.loc[:, key].as_matrix()
C:\test\mucinous_training\TCGA_mucinous_DNN_space\codes\lib\dataProcess.py:134: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  xdata = data.iloc[:, x_start:x_end].as_matrix()
c:\users\hgh97\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [21]:
print("train data: "+str(train_x.shape))
print("test data: "+str(test_x.shape))
print("clin data: "+str(cli_x.shape))

train data: (1452, 561)
test data: (82, 561)
clin data: (10, 561)


In [22]:
if train_x.shape[1] == test_x.shape[1] and test_x.shape[1] == cli_x.shape[1]:
    input_dim = train_x.shape[1]
    print("################ input dim is "+str(input_dim)+" ################")
else:
    print("dimensions are not equal. please check your dataset")

################ input dim is 561 ################


In [23]:
#geneset_merged=keras.layers.Input(shape=(input_dim,))

# 2. Establishing & training/testing the DNN model

## 1) Establishing DNN model

In [24]:
nodes_list = [[500,300,100]]
        
input_m = keras.layers.Input(shape=(input_dim,))

h1_m3 = keras.layers.Dense(500,activation="relu")(input_m)
h1_d = keras.layers.Dropout(0.3,)(h1_m3)
h2_m3 = keras.layers.Dense(300,activation="relu")(h1_d)
h2_d = keras.layers.Dropout(0.3,)(h2_m3)
h3_m3 = keras.layers.Dense(100,activation="relu")(h2_d)
h3_d = keras.layers.Dropout(0.3,)(h3_m3)
                
early_stopping = EarlyStopping(patience = 10)


In [25]:
predictions = Dense(6, activation='softmax', name='predictions')(h3_d)
model1 = Model(inputs = input_m, output = predictions)
adam = keras.optimizers.Adam()
model1.compile(optimizer=adam, loss ='categorical_crossentropy', metrics=['accuracy'])
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 561)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 500)               281000    
_________________________________________________________________
dropout_4 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 300)               150300    
_________________________________________________________________
dropout_5 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               30100     
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
__________

c:\users\hgh97\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  


## 2) Training & Testing the DNN model

In [26]:
#plot_model(model, to_file='model.png')
model1.fit(x=train_x, y=train_y1,callbacks=[early_stopping], epochs = 50,batch_size=25, verbose=0)
print("train acc: "+str(model1.evaluate(train_x,train_y1, verbose=0)[1])+", test acc: "+str(model1.evaluate(test_x,test_y1, verbose=0)[1]))
print("=============================================Complete Train=================================")

c:\users\hgh97\appdata\local\programs\python\python35\lib\site-packages\keras\callbacks.py:535: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


[0.9116783, 0.9268293]
=============================================Complete Test=================================


In [1]:
print(save_model_path+limma_option+method+"_"+geneset+"_model.h5")
model1.save(save_model_path+limma_option+"_"+method+"_"+geneset+"_model.h5")

NameError: name 'save_model_path' is not defined

## 3) Predict train & test & clinical dataset

In [28]:
train_h=model1.predict(train_x, batch_size=None, verbose=0, steps=None)
test_h=model1.predict(test_x, batch_size=None, verbose=0, steps=None)
cli_h=model1.predict(cli_x, batch_size=None, verbose=0, steps=None)

In [29]:
train_p=[]
for i in range(len(train_h)):
    train=train_h[i].tolist()
    train_p.append(train.index(max(train)))

In [30]:
test_p=[]
for i in range(len(test_h)):
    test=test_h[i].tolist()
    test_p.append(test.index(max(test)))

In [31]:
cli_p=[]
for i in range(len(cli_h)):
    cli=cli_h[i].tolist()
    cli_p.append(cli.index(max(cli)))

## 4) Making result table

In [32]:
df_train_h=df(train_h)
df_train_h.columns=['CESC','COAD','PAAD','STAD','UCEC','UCS']

df_train_p=df(train_p)
df_train_p.columns=['prediction']

df_train_y=df(train_y)
df_train_y.columns=['y']

pd.concat([train_data['sample'],df_train_h,df_train_p,df_train_y],axis=1).to_csv(save_result_path+limma_option+"_nonmucinous_"+method+"_"+geneset+"_train_result.csv",index=False)

In [33]:
df_test_h=df(test_h)
df_test_h.columns=['CESC','COAD','PAAD','STAD','UCEC','UCS']

df_test_p=df(test_p)
df_test_p.columns=['prediction']

df_test_y=df(test_y)
df_test_y.columns=['y']

pd.concat([test_data['sample'],df_test_h,df_test_p,df_test_y],axis=1).to_csv(save_result_path+limma_option+"_mucinous_"+method+"_"+geneset+"_test_result.csv",index=False)

In [34]:
df_cli_h=df(cli_h)
df_cli_h.columns=['CESC','COAD','PAAD','STAD','UCEC','UCS']

df_cli_p=df(cli_p)
df_cli_p.columns=['prediction']

pd.concat([cli_data['sample'],df_cli_h,df_cli_p],axis=1).to_csv(save_result_path+limma_option+"_clin_"+method+"_"+geneset+"_clin_result.csv",index=False)